## LSTM Stockprice Demo

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, LSTM,Input
from livelossplot import PlotLossesKeras
from sklearn.preprocessing import MinMaxScaler
import keras

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from livelossplot import PlotLossesKeras

In [ ]:
dataset_train = pd.read_csv('TataGlobal_train.csv')
dataset_test = pd.read_csv('TataGlobal_test.csv')
training_set = dataset_train.iloc[:, 1:2].values
test_set = dataset_test.iloc[:, 1:2].values
training_set

len(dataset_train),len(dataset_test)

In [ ]:
#dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)

In [ ]:
lookback=60
new_open=[]
for i in range(0,len(dataset_train)-lookback):
    new_open.append(dataset_train.Open[i+lookback])
for i in range(0,lookback):
    new_open.append(0)
print(len(dataset_train),len(new_open))

In [ ]:
dataset_train['new_open']=new_open

In [ ]:
dataset_train[['Date','Open','new_open']].tail(65)

In [ ]:
features=dataset_train['Open']
target=dataset_train['new_open']

In [ ]:
features=features.to_numpy().reshape(-1,1)
target=target.to_numpy().reshape(-1,1)


In [ ]:
test_set.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
features = sc.fit_transform(features)
target=sc.fit_transform(target)


In [ ]:
testset=sc.fit_transform(test_set)

In [ ]:
target=target.reshape(-1,)

In [ ]:
features.shape,target.shape

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(features,target,test_size=0.2,random_state=879)

In [ ]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

In [ ]:
xtrain=xtrain.reshape(xtrain.shape[0],xtrain.shape[1],1)
xtest=xtest.reshape(xtest.shape[0],xtest.shape[1],1)

In [ ]:
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

In [ ]:
model = Sequential()
model.add(LSTM(1000,input_shape=(xtrain.shape[1:])))
#model.add(LSTM(units=50))
#model.add(LSTM(units=50))
#model.add(LSTM(units=50))
#model.add(Dropout(0.2))
model.add(Dense(1))


In [ ]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=(['accuracy']))
history=model.fit(xtrain,ytrain,epochs=100,batch_size=32,verbose=1,callbacks=[PlotLossesKeras()],validation_data=(xtest,ytest))

### Test Data

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
testset=testset.reshape(testset.shape[0],testset.shape[1],1)

In [ ]:
predict_values=model.predict(testset)

In [ ]:
predict=sc.inverse_transform(predict_values)

In [ ]:
train_predict=model.predict(xtrain)

In [ ]:
train_predict=sc.inverse_transform(train_predict)

In [ ]:
dataset_test['predicted_values']=predict

In [ ]:
plt.plot(dataset_test.Open)
plt.plot(dataset_test.predicted_values)